In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# --- 1. Load data ---
file_path = "/data/faiss/books_data.csv"
df = pd.read_csv(file_path)

df['content'] = (
    df['name'].fillna('') + ' ' +
    df['authors'].fillna('') + ' ' +
    df['category'].fillna('') + ' ' +
    df['short_description'].fillna('')
)

In [4]:
# --- 2. Load model ---
model = SentenceTransformer('keepitreal/vietnamese-sbert')

In [5]:
# --- 3. Encode embeddings ---
print("🔄 Đang sinh embedding...")
embeddings = model.encode(df['content'], convert_to_tensor=False, show_progress_bar=True)
title_embeddings = model.encode(df['name'].fillna('').tolist(), convert_to_tensor=False, show_progress_bar=True)
authors_embeddings = model.encode(df['authors'].fillna('').tolist(), convert_to_tensor=False, show_progress_bar=True)
category_embeddings = model.encode(df['category'].fillna('').tolist(), convert_to_tensor=False, show_progress_bar=True)

cosine_sim = cosine_similarity(embeddings)

🔄 Đang sinh embedding...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

In [6]:
# --- 4. Phát hiện loại query ---
def detect_query_type(query):
    query_emb = model.encode(query, convert_to_tensor=False)

    sim_title = cosine_similarity([query_emb], title_embeddings)[0].max()
    sim_author = cosine_similarity([query_emb], authors_embeddings)[0].max()
    sim_category = cosine_similarity([query_emb], category_embeddings)[0].max()
    sim_content = cosine_similarity([query_emb], embeddings)[0].max()

    sims = {
        "title": sim_title,
        "author": sim_author,
        "category": sim_category,
        "content": sim_content
    }
    query_type = max(sims, key=sims.get)
    return query_type, sims[query_type]


In [7]:
# --- 5. Gộp các loại embedding ---
def find_best_match_across_embeddings(query_text):
    query_embedding = model.encode(query_text, convert_to_tensor=False)
    sim_title = cosine_similarity([query_embedding], title_embeddings)[0]
    sim_content = cosine_similarity([query_embedding], embeddings)[0]
    sim_author = cosine_similarity([query_embedding], authors_embeddings)[0]
    sim_category = cosine_similarity([query_embedding], category_embeddings)[0]

    combined_sims = (
        0.25 * sim_title +
        0.25 * sim_content +
        0.25 * sim_author +
        0.25 * sim_category
    )
    top_idx = np.argmax(combined_sims)
    top_score = combined_sims[top_idx]
    return top_idx, top_score

In [41]:
# --- 6. Gợi ý sách ---
def recommend_books(query_text, top_n=5, threshold=0.5):
    query_text = query_text.strip()
    
    #  Xác định loại query (author / category / title / content)
    query_type, confidence = detect_query_type(query_text)
    print(f" Nhận diện query thuộc loại: {query_type} (độ tin cậy: {confidence:.2f})")

    #  Trường hợp query là tác giả
    if query_type == "author":
        idx = np.argmax(cosine_similarity([model.encode(query_text)], authors_embeddings)[0])
        author_name = df.loc[idx, 'authors']
        print(f" Gợi ý các sách của tác giả: {author_name}")
        result = df[df['authors'].str.contains(author_name, case=False, na=False)]

    #  Trường hợp query là thể loại
    elif query_type == "category":
        idx = np.argmax(cosine_similarity([model.encode(query_text)], category_embeddings)[0])
        category_name = df.loc[idx, 'category']
        print(f" Gợi ý các sách thuộc thể loại: {category_name}")
        result = df[df['category'].str.contains(category_name, case=False, na=False)]

    #  Còn lại (title hoặc content)
    else:
        #  Kiểm tra khớp trực tiếp tên sách (ưu tiên nhất)
        matches = df[df['name'].str.contains(query_text, case=False, na=False)]
        if not matches.empty:
            print(f" Khớp chính xác theo tên sách: {matches.iloc[0]['name']}")
            result = matches[['name', 'authors', 'category', 'link']].head(top_n).copy()
            return result
            
        print(" Truy vấn được xem là liên quan đến tiêu đề hoặc nội dung.")
        query_emb = model.encode(query_text, convert_to_tensor=False)

        # Tính tương đồng dựa trên content tổng hợp (tên + mô tả)
        sim_content = cosine_similarity([query_emb], embeddings)[0]

        top_idx = np.argmax(sim_content)
        top_score = sim_content[top_idx]

        if top_score < threshold:
            print(" Không có match tốt — gợi ý ngẫu nhiên.")
            result = df.sample(top_n)
        else:
            print(f" Tìm thấy sách gần nhất: {df.loc[top_idx, 'name']} (similarity: {top_score:.2f})")

            # Lấy top N sách tương tự
            sim_scores = list(enumerate(sim_content))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n + 1]
            result = df.iloc[[i[0] for i in sim_scores]]

    # Chuẩn hóa kết quả
    result = result[['name', 'authors', 'category', 'link']].head(top_n).copy()
    return result


In [42]:
# --- 7. Test ---
if __name__ == "__main__":
    query = input("Nhập nội dung/tên/tác giả/thể loại: ")
    recs = recommend_books(query, top_n=5)
    print("\n Top sách gợi ý:\n")
    print(recs.to_string(index=False))

Nhập nội dung/tên/tác giả/thể loại:  tam quốc


 Nhận diện query thuộc loại: title (độ tin cậy: 0.48)
 Khớp chính xác theo tên sách: Tam Quốc Diễn Nghĩa - Bộ 3 Tập (Tái bản )

 Top sách gợi ý:

                                                                 name       authors                      category                                                   link
                            Tam Quốc Diễn Nghĩa - Bộ 3 Tập (Tái bản ) La Quán Trung Tác Phẩm Kinh Điển Nước Ngoài     https://tiki.vn/product-p9729890.html?spid=9729891
                                 Tam Quốc Diễn Nghĩa (Trọn Bộ 04 Tập) La Quán Trung Tác Phẩm Kinh Điển Nước Ngoài     https://tiki.vn/product-p568762.html?spid=17509940
                                      Tam Quốc Diễn Nghĩa (Huy Hoàng) La Quán Trung Tác Phẩm Kinh Điển Nước Ngoài     https://tiki.vn/product-p542586.html?spid=20452266
                                  Tam Quốc Lầy Lội Diễn Nghĩa - Tập 1      Trần Lôi                          Root https://tiki.vn/product-p275339961.html?spid=275339962
Combo Tam